# Day-8 GNSS Error Prediction Training Notebook
This notebook walks through the full workflow to train and evaluate the hybrid LSTM/Transformer pipeline for 15-minute cadence Day-8 forecasts.

## 1. Environment & Dependencies
Use the project's virtual environment, then install the additional packages listed below if they are missing.

In [ ]:
# Install dependencies if needed (uncomment the next line on first run)
# !pip install -r ..\requirements.txt --upgrade
import os
import json
from pathlib import Path
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## 2. Data Ingestion & QA
Load raw CSV files, normalise column names, parse timestamps, and resample to a strict 15-minute grid.

In [ ]:
RAW_DATA_DIR = Path('..') / 'backend' / 'data'
INPUT_GLOB = '*.csv'
FEATURES = ['x_error', 'y_error', 'z_error', 'satclockerror']
TIMESTAMP_COL = 'utc_time'
TARGET_HORIZON = 96  # 24h @ 15 minutes
SPEED_OF_LIGHT = 299_792_458.0
SATCLOCK_BOUNDS = (-4.0, 4.0)

def load_dataset(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path)
    df.columns = df.columns.str.strip().str.lower()
    df = df.rename(columns={'x_error (m)': 'x_error', 'y_error (m)': 'y_error', 'z_error (m)': 'z_error', 'satclockerror (m)': 'satclockerror', 'time': 'utc_time'})
    df[TIMESTAMP_COL] = pd.to_datetime(df[TIMESTAMP_COL], errors='coerce')
    df = df.dropna(subset=[TIMESTAMP_COL]).sort_values(TIMESTAMP_COL).reset_index(drop=True)
    for c in FEATURES:
        df[c] = pd.to_numeric(df[c], errors='coerce')
    df = df.dropna(subset=FEATURES)
    return df

raw_files = list(RAW_DATA_DIR.glob(INPUT_GLOB))
print(f'Found {len(raw_files)} raw files')
raw_files[:3]

In [ ]:
def regularise_to_15min(df: pd.DataFrame) -> pd.DataFrame:
    idx = pd.date_range(df[TIMESTAMP_COL].min(), df[TIMESTAMP_COL].max(), freq='15T')
    df = df.set_index(TIMESTAMP_COL).reindex(idx)
    df[FEATURES] = df[FEATURES].interpolate().ffill().bfill()
    df = df.reset_index().rename(columns={'index': TIMESTAMP_COL})
    return df

def normalise_satclock(df: pd.DataFrame) -> tuple[pd.DataFrame, str]:
    sat = pd.to_numeric(df['satclockerror'], errors='coerce')
    unit = 'seconds'
    if sat.abs().median() > 1_000:
        df['satclockerror'] = sat / SPEED_OF_LIGHT
        unit = 'meters'
    df['satclockerror'] = df['satclockerror'].clip(*SATCLOCK_BOUNDS)
    df['satclockerror'] = df['satclockerror'].fillna(method='ffill').fillna(method='bfill')
    return df, unit

cleaned_frames = []
for path in raw_files:
    df = load_dataset(path)
    df = regularise_to_15min(df)
    df, unit = normalise_satclock(df)
    df['source_file'] = path.name
    df['satclock_unit'] = unit
    cleaned_frames.append(df)

dataset = pd.concat(cleaned_frames, ignore_index=True)
dataset.head()

## 3. Feature Engineering
Create lag features, rolling statistics, harmonic time encodings, and optional RTN transforms.

In [ ]:
def add_temporal_features(df: pd.DataFrame) -> pd.DataFrame:
    ts = df[TIMESTAMP_COL]
    df['hour'] = ts.dt.hour + ts.dt.minute / 60.0
    df['sin24'] = np.sin(2 * np.pi * df['hour'] / 24.0)
    df['cos24'] = np.cos(2 * np.pi * df['hour'] / 24.0)
    df['dayofweek'] = ts.dt.dayofweek
    return df

def add_lagged_features(df: pd.DataFrame, lags: list[int]) -> pd.DataFrame:
    for lag in lags:
        for feature in FEATURES:
            df[f'{feature}_lag_{lag}'] = df[feature].shift(lag)
    return df

def add_rolling_stats(df: pd.DataFrame, windows: list[int]) -> pd.DataFrame:
    for window in windows:
        for feature in FEATURES:
            df[f'{feature}_roll_{window}_mean'] = df[feature].rolling(window).mean()
            df[f'{feature}_roll_{window}_std'] = df[feature].rolling(window).std()
    return df

feature_df = add_temporal_features(dataset.copy())
feature_df = add_lagged_features(feature_df, lags=[1, 2, 4, 8, 16, 24, 48])
feature_df = add_rolling_stats(feature_df, windows=[4, 12, 48, 96])
feature_df = feature_df.dropna().reset_index(drop=True)
feature_df.head()

## 4. Windowing
Convert the feature table into (input, output) windows for seq2seq training.

In [ ]:
SEQ_LEN = 96  # 24 hours history
OUT_LEN = 96  # 24 hours forecast

feature_columns = [c for c in feature_df.columns if c not in {TIMESTAMP_COL, 'source_file', 'satclock_unit'}]

def create_windows(values: np.ndarray, seq_len: int, out_len: int) -> tuple[np.ndarray, np.ndarray]:
    Xs, ys = [], []
    total = len(values)
    for start in range(0, total - seq_len - out_len + 1):
        end = start + seq_len
        out_end = end + out_len
        Xs.append(values[start:end])
        ys.append(values[end:out_end, :len(FEATURES)])
    return np.array(Xs), np.array(ys)

values = feature_df[feature_columns].values.astype(np.float32)
X, y = create_windows(values, SEQ_LEN, OUT_LEN)
print(f'Windowed dataset: X={X.shape}, y={y.shape}')

## 5. Train/Validation Split
Use a chronological split (last window block for validation).

In [ ]:
split_index = int(len(X) * 0.8)
X_train, X_val = X[:split_index], X[split_index:]
y_train, y_val = y[:split_index], y[split_index:]
print(f'Train windows: {X_train.shape}, Validation windows: {X_val.shape}')

## 6. LSTM Baseline
Train a two-layer LSTM with Huber loss and early stopping.

In [ ]:
def build_lstm(seq_len: int, n_features: int, out_steps: int) -> keras.Model:
    inputs = keras.Input(shape=(seq_len, n_features))
    x = keras.layers.LSTM(256, return_sequences=True)(inputs)
    x = keras.layers.Dropout(0.2)(x)
    x = keras.layers.LSTM(128)(x)
    x = keras.layers.Dense(128, activation='relu')(x)
    outputs = keras.layers.Dense(out_steps * len(FEATURES))(x)
    outputs = keras.layers.Reshape((out_steps, len(FEATURES)))(outputs)
    model = keras.Model(inputs, outputs, name='lstm_baseline')
    model.compile(optimizer=keras.optimizers.Adam(1e-3), loss='huber', metrics=['mae'])
    return model

lstm_model = build_lstm(SEQ_LEN, X_train.shape[-1], OUT_LEN)
lstm_history = lstm_model.fit(
    X_train,
lstm_history.history.keys()

## 7. Transformer Model
Lightweight encoder-decoder self-attention model for long horizon forecasting.

In [ ]:
def transformer_block(inputs: keras.layers.Layer, num_heads: int, ff_dim: int, dropout: float) -> keras.layers.Layer:
    attn = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=inputs.shape[-1])(inputs, inputs)
    attn = keras.layers.Dropout(dropout)(attn)
    out1 = keras.layers.LayerNormalization(epsilon=1e-6)(inputs + attn)
    ffn = keras.layers.Dense(ff_dim, activation='relu')(out1)
    ffn = keras.layers.Dense(inputs.shape[-1])(ffn)
    ffn = keras.layers.Dropout(dropout)(ffn)
    return keras.layers.LayerNormalization(epsilon=1e-6)(out1 + ffn)

def build_transformer(seq_len: int, n_features: int, out_steps: int) -> keras.Model:
    inputs = keras.Input(shape=(seq_len, n_features))
    x = keras.layers.Dense(128)(inputs)
    for _ in range(3):
        x = transformer_block(x, num_heads=4, ff_dim=256, dropout=0.1)
    x = keras.layers.GlobalAveragePooling1D()(x)
    x = keras.layers.Dense(256, activation='relu')(x)
    outputs = keras.layers.Dense(out_steps * len(FEATURES))(x)
    outputs = keras.layers.Reshape((out_steps, len(FEATURES)))(outputs)
    model = keras.Model(inputs, outputs, name='transformer_horizon')
    model.compile(optimizer=keras.optimizers.Adam(1e-4), loss='huber', metrics=['mae'])
    return model

transformer_model = build_transformer(SEQ_LEN, X_train.shape[-1], OUT_LEN)
transformer_history = transformer_model.fit(
    X_train,
transformer_history.history.keys()

## 8. Evaluation
Compute MAE, RMSE, SISRE proxies, and residual normality checks.

In [ ]:
from sklearn.metrics import mean_absolute_error

def evaluate_model(name: str, model: keras.Model, X_eval: np.ndarray, y_eval: np.ndarray) -> dict:
    preds = model.predict(X_eval, verbose=0)
    preds = preds.reshape(-1, len(FEATURES))
    truth = y_eval.reshape(-1, len(FEATURES))
    metrics = {}
    for i, feature in enumerate(FEATURES):
        metrics[f'{feature}_mae'] = mean_absolute_error(truth[:, i], preds[:, i])
        metrics[f'{feature}_rmse'] = np.sqrt(np.mean((truth[:, i] - preds[:, i]) ** 2))
    metrics['model'] = name
    return metrics

lstm_metrics = evaluate_model('lstm', lstm_model, X_val, y_val)
transformer_metrics = evaluate_model('transformer', transformer_model, X_val, y_val)
lstm_metrics, transformer_metrics

## 9. Ensemble & Model Selection
Weight each model by inverse validation MAE to pick the best performer for each prediction.

In [ ]:
def select_model(step_index: int, gap_seconds: float) -> str:
    if gap_seconds >= 3600:
        return 'transformer'
    return 'lstm'

selection_example = select_model(step_index=12, gap_seconds=1800)
selection_example

## 10. Export Artifacts
Save the scaler and best-performing weights for deployment.

In [ ]:
MODEL_DIR = Path('..') / 'backend' / 'models'
MODEL_DIR.mkdir(exist_ok=True, parents=True)

np.save(MODEL_DIR / 'feature_columns.npy', np.array(feature_columns))
joblib.dump(StandardScaler().fit(values), MODEL_DIR / 'scaler.pkl')
lstm_model.save(MODEL_DIR / 'lstm_model.h5', include_optimizer=False)
transformer_model.save(MODEL_DIR / 'transformer_model.h5', include_optimizer=False)

## 11. Day-8 Inference
Use the trained models to generate Day-8 predictions for a selected week.

In [ ]:
from backend.predict import predict_from_dataframe  # noqa: E402

sample_week = dataset[dataset['source_file'] == raw_files[0].name]
prediction_result = predict_from_dataframe(sample_week)
json.dumps(prediction_result['prediction_summary'], indent=2)